In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Note:** Vì yêu cầu submit của challenge đã kết thúc là không được sử dụng '+Add data' dữ liệu từ bên ngoài nên notebook này em sẽ chỉ thực hiện lại các bước xử lý từ đầu đến cuối để có thể nộp bài, còn phần EDA và phần Feature Extraction em đã thực hiện ở 2 notebook trước đó ạ.

---------------------------------------------------------------------------------------------------------------------------------------------------------

> # Xử lý dữ liệu và chạy model cuối

(Các block code này em đã thực hiện ở 2 notebook trước rồi ạ)

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import nltk
import pickle
import zipfile
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import word_tokenize

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

In [ ]:
stop_words = stopwords.words('english')
wordnet = WordNetLemmatizer() 

In [ ]:
# Xóa bỏ các từ là stopword
def delStopwords(text):
    return ' '.join([w for w in word_tokenize(text) if not w in stop_words])

# Đưa các từ về dạng từ gốc của nó
def lemmatizeText(text):
    return ' '.join([wordnet.lemmatize(w, pos='v') for w in word_tokenize(text)])

In [ ]:
def clean_text(text):
    cleaned_text = re.sub('[\n]',' ',text)    #xóa các ký tự xuống dòng
    cleaned_text = re.sub('[A-Z]+', lambda m: m.group(0).lower(), cleaned_text)    #lower các ký tự 
    cleaned_text = re.sub('[^a-zA-Z]',' ',cleaned_text).strip()    #xóa các ký tự không phải là chữ cái
    cleaned_text = delStopwords(cleaned_text)    #xóa stopwords
    cleaned_text = lemmatizeText(cleaned_text)    #lemmatize các từ về danh noun
    return cleaned_text

In [ ]:
train_text = []
target = []
for i in range(len(train)):
    train_text.append(clean_text(train['question_text'][i]))
    target.append((train['target'][i]).astype('int32'))
    
test_text = []
for i in range(len(test)):
    test_text.append(clean_text(test['question_text'][i]))

In [ ]:
len(train_text), len(target), len(test_text)

In [ ]:
with zipfile.ZipFile('../input/quora-insincere-questions-classification/embeddings.zip', 'r') as zip_ref:
    zip_ref.extractall('./embeddings')

In [ ]:
train_text = np.array(train_text)
test_text = np.array(test_text)

type(train_text), type(test_text), type(target)

In [ ]:
MAX_LEN = 30    # Độ dài của 1 câu 
EMBEDDING_SIZE = 300    # Số chiều embedding khi sử dụng fastText mặc định là (300,) 

In [ ]:
from gensim.models import FastText 

def load_fasttext():
    print('loading word embeddings...')
    embeddings_index = {}
    f = open('./embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec',encoding='utf-8')
    for line in f:
        values = line.strip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))

    return embeddings_index

In [ ]:
embeddings_index = load_fasttext()

In [ ]:
all_text = list(train_text)+list(test_text)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

train_padded_sequences = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [ ]:
word_index = tokenizer.word_index
num_words = len(word_index)

In [ ]:
def prepare_matrix(embedding_index, emb_size = EMBEDDING_SIZE):
    embedding_matrix = np.zeros((num_words+1, emb_size))

    for word, i in word_index.items():
        if i > num_words:
            continue

        emb_vec = embedding_index.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

    return embedding_matrix

In [ ]:
embedding_matrix = prepare_matrix(embeddings_index)
embedding_matrix.shape

In [ ]:
LEARNING_RATE = 0.001
BATCH_SIZE = 512
EPOCHS = 10

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def plot_train_val_f1score(hist):
    plt.figure(figsize=(12,8))
    epoch_list = list(range(1,len(hist.history['f1'])+1))
    plt.plot(epoch_list, hist.history['f1'],label='f1')
    plt.plot(epoch_list, hist.history['val_f1'],label='val_f1')
    plt.xlabel('epoches')
    plt.ylabel('score')
    plt.legend()
    plt.show()

def tweak_threshold(pred_prob, truth):
    thresholds = []
    scores = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        thresholds.append(thresh)
        score = f1_score(truth, (pred_prob>thresh).astype(int))
        scores.append(score)
    return np.max(scores), thresholds[np.argmax(scores)]

Sau một vài lần chạy thử với việc thay đổi các tham số cùng với việc chạy StratifiedKFold ở notebook trước,  em đã chọn ra mô hình tốt nhất mình đạt được

In [ ]:
def build_model(embedding_matrix, num_words, EMBEDDING_SIZE):
    inp = Input(shape=(MAX_LEN,))
    x = Embedding(num_words+1, EMBEDDING_SIZE, weights=[embedding_matrix], trainable=False)(inp)
    x = Dropout(0.3)(x)
    x1 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(GRU(256, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = Adam(lr=LEARNING_RATE)
    model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy',f1])
    return model

In [ ]:
es = EarlyStopping(monitor=f1, mode='max', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, min_=0.0005, verbose=1)
class_weights = {0: 1, 1: 5}

In [ ]:
model = build_model(embedding_matrix, num_words, EMBEDDING_SIZE)

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(train_padded_sequences, np.array(target), test_size=0.2, shuffle=True, stratify=target)

In [ ]:
history = model.fit(X_tr, y_tr,
                   batch_size=BATCH_SIZE, 
                   epochs=EPOCHS, 
                   validation_data=(X_val,y_val), 
                   callbacks=[es, reduce_lr],
                   class_weight=class_weights,
                   verbose=1
                   )

In [ ]:
plot_train_val_f1score(history)

Kết quả thu được là mô hình không còn bị overfit hay underfit

In [ ]:
val_pred = np.squeeze(model.predict([X_val], batch_size=BATCH_SIZE))
f1score, threshold = tweak_threshold(val_pred, np.squeeze(y_val))
print(f"F1score: {round(f1score, 4)} for threshold: {threshold} on validation data")
val_prediction = (val_pred > threshold).astype(int)
print('Classification Report on validation data: \n', classification_report(y_val, val_prediction))

Tuy nhiên, khi chạy với model mới kết quả thu được cho precision của nhãn '1' vẫn là khá thấp, chỉ 51%. Điều này có thể là do mô hình em sử dụng vẫn còn đơn giản nên chưa trích chọn được các feature tốt nhất. Em xin phép tiếp tục cải thiện thêm mô hình của mình sau ạ.

----------------------------------------------------------------------------------------------------------------------------------------------------------

> # Dự đoán trên tập test.csv và nộp bài

In [ ]:
y_test = np.squeeze(model.predict([test_padded_sequences], batch_size=BATCH_SIZE))

In [ ]:
label_test = (y_test > threshold).astype(int)
pd.DataFrame(label_test).value_counts()

In [ ]:
output = pd.DataFrame({'qid': test['qid'].values})
output['prediction'] = label_test
output.to_csv("submission.csv", index=False)

In [ ]:
model.save('model_Bidirectional_LSTM_GRU.h5')

---------------------------------------------------------------------------------------------------------------------------------------------

> # Summary

Kết quả submit kaggle: 0.62463  

Những gì đã học được:  
- Quá trình EDA cũng như là việc xây dựng các bước tiền xử lý từ EDA.
- Xử lý bài toán với dữ liệu imbalance.
- Học máy cơ bản với các kiến thức nền bắt đầu từ khái niệm, các hàm loss, compile model, fit model, ...
- Các kiến thức liên quan đến mạng học sâu như CNN, RNN, LSTM, GRU cũng như là set up các tham số cho mạng.
- Các cách nhận biết, đánh giá model học sâu và cách tinh chỉnh model sao cho đạt kết quả tốt nhất.

Những gì cần cải thiện trong tương lai:
- Xây dựng mô hình phức tạp hơn để có thể học cũng như dự đoán tốt hơn cho nhãn thiểu số.
- Áp dụng cơ chế Attention cho bài toán dạng input sequences như challenge này để tập trung vào các đặc trưng tốt nhất.
- Thay đổi các bước tiền xử lý vì như đã tham khảo thì có vẻ rằng em đã tiền xử lý quá kĩ khi áp dụng các pre-trained word embedding.

------------------------------------------------------------------------------------------------------------------------------------------------------------